In [1]:
from src.downloadDataset import DownloadCEMSDataset as CemsDataset

import warnings
warnings.filterwarnings('ignore')

download_cems = CemsDataset()

"""Uncomment this line if you would download and create the whole dataset"""
# download_cems.download_EMSR_Manager(allActivation = True)

"""Uncomment this line if you would download a specif activation, pay attention to the format: EMSRXXX and AOIXX"""
download_cems.download_EMSR_Manager(Emsr = "EMSR638", Aoi = "AOI01")

"""Uncomment this line if you would create/recreate the cloud cover for the whole dataset"""
# download_cems.downloadCloudCover()

"""Uncomment this line if you would re/download land cover for the whole dataset"""
# download_cems.downloadLandCover()

"""Uncomment this line if you would re/save satellite data with all information about the dataset"""
# download_cems.saveSatelliteDataCSV()

"""Uncomment this line if you have changed the folder arrangement, so it will be possible replicate the dataset arrangement
NOTE: USE THIS FUNCTION ONLY IN CASE OF DEBUG"""
# download_cems.createPreconfiguredCSV()

"""Uncomment this line if you would copy all .png images in a separate folder"""
# download_cems.copyAllImageToRGBFolder()


Download started...
Download EMSR638_AOI01


Warning 1: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.
Lightning automatically upgraded your loaded checkpoint from v1.5.10 to v1.9.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file ../../.config/maskaymodels/unetmobv2.ckpt`
100%|██████████| 8/8 [00:40<00:00,  5.04s/it]
Warning 1: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.


Warning 1: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.
Lightning automatically upgraded your loaded checkpoint from v1.5.10 to v1.9.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file ../../.config/maskaymodels/unetmobv2.ckpt`
100%|██████████| 8/8 [00:39<00:00,  4.94s/it]
Warning 1: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.


Raster mask completed
Writing cloudCoverage.csv....
Activation EMSR638_AOI01 done!!


'Uncomment this line if you would copy all .png images in a separate folder'

In [2]:
"""try to measure the similarity between various pre-fire images and the post-fire image"""

from skimage import io
from skimage.color import rgb2lab
import numpy as np
import os
import json
from datetime import datetime, timedelta

import shapely
from shapely.geometry import shape, GeometryCollection

from src.downloadDataset import DownloadCEMSDataset as CemsDataset
import src.utils_variable as utils

import warnings
warnings.filterwarnings('ignore')

download_cems = CemsDataset()

# def spectral_matching(reference_image_path, comparison_image_paths):
#     # Load the reference image
#     reference_image = io.imread(reference_image_path)
    
#     # Convert the reference image to the LAB color space
#     reference_image_lab = rgb2lab(reference_image)
    
#     # Initialize an empty list to store the similarity scores
#     similarity_scores = []
    
#     # Loop through the comparison images
#     for comparison_image_path in comparison_image_paths:
#         # Load the comparison image
#         comparison_image = io.imread(comparison_image_path)
        
#         # Convert the comparison image to the LAB color space
#         comparison_image_lab = rgb2lab(comparison_image)
        
#         # Calculate the Euclidean distance between the reference and comparison images
#         distance = np.linalg.norm(reference_image_lab - comparison_image_lab)
        
#         # Add the similarity score to the list
#         similarity_scores.append(distance)
        
#     return similarity_scores

def spectral_matching(reference_image_path, comparison_image_paths):
    # Load the reference image
    reference_image = io.imread(reference_image_path)
    
    # Extract the reflectance values from all 12 channels of the reference image
    reference_image_bands = reference_image[:, :, :12]
    
    # Initialize an empty list to store the similarity scores
    similarity_scores = []
    
    # Loop through the comparison images
    for comparison_image_path in comparison_image_paths:
        # Load the comparison image
        comparison_image = io.imread(comparison_image_path)
        
        # Extract the reflectance values from all 12 channels of the comparison image
        comparison_image_bands = comparison_image[:, :, :12]
        
        # Calculate the Euclidean distance between the reference and comparison images
        distance = np.linalg.norm(reference_image_bands - comparison_image_bands)
        
        # Add the similarity score to the list
        similarity_scores.append(distance)
        
    return similarity_scores



In [4]:
Emsr = "EMSR632"
Aoi = "AOI01"
with open(utils.copernicusFolderPath + utils.dateJson) as json_file:
        dataDict = json.load(json_file)

with open(utils.copernicusFolderPath + Emsr + "/" + Aoi + "/GRA/PRODUCT/" + Emsr + "_" + Aoi + "_GRA_PRODUCT_areaOfInterestA.json") as f:
        features = json.load(f)["features"]
                
# NOTE: buffer(0) is a trick for fixing scenarios where polygons have overlapping coordinates 
polygon_AOI = GeometryCollection([shape(feature["geometry"]).buffer(0) for feature in features if "type" in list(feature["geometry"]) and feature["geometry"]["type"] == "Polygon"  ]  )
multipolygon_AOI = shapely.geometry.MultiPolygon(polygon_AOI)

for i in range(0, 30, 5) :
    
    end_date_ESMR = datetime.strptime(dataDict.get(Emsr), "%Y-%m-%dT%H:%M:%S.%f") - timedelta(days = i)
    start_date_ESMR = end_date_ESMR - timedelta(days = i)

    files, aqcuisition = download_cems.retrieve_images_from_sentinel("prova/", multipolygon_AOI, start_date_ESMR, end_date_ESMR, Emsr, Aoi)

    print(files)
    # print(aqcuisition)

    # for folder, _, filenames in os.walk(imagesELIMINA.data_folder):
    #     for filename in filenames:
    #         print(os.path.join(folder, filename))

paths = []
for dirpath, dirnames, filenames in os.walk("prova/"):
    for filename in [f for f in filenames if ".tiff" in f]:
        paths.append(dirpath + "/" + filename)

pathEMSR = utils.output_folder + "dataOptimal/EMSR632/AOI01/EMSR632_AOI01_01/EMSR632_AOI01_01_S2L2A.tiff"

results = spectral_matching(pathEMSR, paths)
print(results)

No images found in period 2022-09-08 00:00:00 - 2022-09-08 00:00:00.
None
['cf62018400b5f75abe3ffcc1e1c6e402/response.tiff']
['aef8b7cc10469e578156af8ac5e96833/response.tiff']
['c112855ec6eb6221ee4233e8f00355c0/response.tiff']
['17da74f95fac2e72c088f55596b29a52/response.tiff']
['a4df88ac42b09e36e90d1d042fcbf0bb/response.tiff']
[221.23926, 232.21922, 232.21922, 203.90402, 221.23926]


In [ ]:



# To find Sentinel-2 images that are similar to a reference image over a certain time interval, 
# you could use a combination of the approaches I mentioned earlier:

# 1 Use a spectral matching algorithm: This approach involves comparing the spectral characteristics of the reference 
# image with those of other Sentinel-2 images. You could use a spectral matching algorithm to identify images with similar 
# spectral profiles to the reference image.

# 2 Use image registration: Image registration is the process of aligning two or more images of the same scene so 
# that they can be compared. You could use image registration to align the reference image with other Sentinel-2 images
# and then compare them to identify those that are most similar to the reference image.

# 3 Use image segmentation: Image segmentation is the process of dividing an image into distinct regions or segments based 
# on certain characteristics. You could use image segmentation to divide the reference image into segments and then compare 
# the segments with those in other Sentinel-2 images to identify those that are most similar.

# 4 Use a machine learning model: You could train a machine learning model to identify Sentinel-2 images that are 
# similar to the reference image. To do this, you would need to have a large dataset of Sentinel-2 images along with 
# labels indicating which images are similar to the reference image. The model could then be used to predict which of the 
# other Sentinel-2 images are most similar to the reference image.

# You could also try using a combination of these approaches to improve the accuracy of your results.
# To find the Sentinel-2 images that you want to download, you can use the Sentinel Hub EO Browser or the Sentinel-2 Toolbox. 
# These tools allow you to search for Sentinel-2 images based on various criteria, including the time interval you are interested in. 
# You can then use the tools to download the images that you have identified as being most similar to your reference image.
    